In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 17


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 16.6 ms, sys: 8.12 ms, total: 24.7 ms
Wall time: 24 ms


## Tokenize the training data set and build a vocabulary from it
Want to do this to minimize the vocab to just what is in the training set for performance.

using defaults:  strip all punctuation except apostrophies, set to lower case and split on spaces. No restriction on length of vocabulary.

In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, x_val_ids, x_test_ids, tokenizer = tokenize_sentences(x_train, x_val, np.array([]), max_len)


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer, embed_dim)

CPU times: user 31.5 s, sys: 103 ms, total: 31.6 s
Wall time: 31.4 s


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


In [12]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [13]:
%%time
# this is very quick, so we can use grid search on these models.

run_logfile = "runs2ballen17.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models  (Removed the filter sizes 64 that exceed the size of the shorter sequence)
for k_sizes in [ [8,16],  [4,8,12], [4,8,16], [8,12,16] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96, 96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal17",
                      model_dir        = "../models/CNNBal17",
                      max_len          = max_len)
            
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 4s 8ms/step - loss: 0.6291 - accuracy: 0.6314 - val_loss: 0.6054 - val_accuracy: 0.6641
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232321/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4324 - accuracy: 0.8026 - val_loss: 0.3790 - val_accuracy: 0.8287
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232321/assets
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.2790 - accuracy: 0.8906 - val_loss: 0.3453 - val_accuracy: 0.8436
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232321/assets
Epoch 4/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1646 - accuracy: 0.9425 - val_loss: 0.1984 - val_accuracy: 0.9241
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232321/assets
Epoch 5/20
288/288 [=====

288/288 [==============================] - 2s 5ms/step - loss: 0.0322 - accuracy: 0.9901 - val_loss: 0.2996 - val_accuracy: 0.9209
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232428/assets
Epoch 8/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0282 - accuracy: 0.9911 - val_loss: 0.2754 - val_accuracy: 0.9286
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232428/assets
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0244 - accuracy: 0.9909 - val_loss: 0.2973 - val_accuracy: 0.9250
Epoch 10/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0171 - accuracy: 0.9949 - val_loss: 0.4193 - val_accuracy: 0.9096
Epoch 11/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0129 - accuracy: 0.9962 - val_loss: 0.3778 - val_accuracy: 0.9132
Epoch 12/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0229 - accuracy: 0.9916 - val_loss: 0.3733 - val_accuracy: 0.9146
Epoch 

INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232536/assets
Epoch 13/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0076 - accuracy: 0.9980 - val_loss: 0.3954 - val_accuracy: 0.9259
Epoch 14/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0083 - accuracy: 0.9969 - val_loss: 0.4026 - val_accuracy: 0.9263
Epoch 15/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0071 - accuracy: 0.9973 - val_loss: 0.3505 - val_accuracy: 0.9286
Epoch 16/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0069 - accuracy: 0.9986 - val_loss: 0.4786 - val_accuracy: 0.9064
Epoch 17/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0129 - accuracy: 0.9948 - val_loss: 0.6122 - val_accuracy: 0.9024
Epoch 18/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0076 - accuracy: 0.9976 - val_loss: 0.4069 - val_accuracy: 0.9331
Epoch 19/20
288/288 [==============================] - 1s 5ms/step 

Epoch 20/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0049 - accuracy: 0.9989 - val_loss: 0.4619 - val_accuracy: 0.9227



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 6ms/step - loss: 0.6458 - accuracy: 0.6271 - val_loss: 0.6234 - val_accuracy: 0.6677
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232716/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4435 - accuracy: 0.7944 - val_loss: 0.5819 - val_accuracy: 0.7369
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232716/assets
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.2711 - accuracy: 0.8884 - val_loss: 0.3596 - val_accuracy: 0.8476
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232716/assets
Epoch 4/20
288/288

288/288 [==============================] - 1s 5ms/step - loss: 0.0381 - accuracy: 0.9884 - val_loss: 0.4154 - val_accuracy: 0.8797
Epoch 7/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0211 - accuracy: 0.9938 - val_loss: 0.2344 - val_accuracy: 0.9367
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232821/assets
Epoch 8/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0191 - accuracy: 0.9941 - val_loss: 0.5746 - val_accuracy: 0.8734
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0156 - accuracy: 0.9949 - val_loss: 0.3321 - val_accuracy: 0.9191
Epoch 10/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0166 - accuracy: 0.9953 - val_loss: 0.2179 - val_accuracy: 0.9498
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-232821/assets
Epoch 11/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0174 - accuracy: 0.9934 - val_loss: 0.4286 - val_accuracy: 0.9114
Epoch 1

...

Epoch 11/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0112 - accuracy: 0.9959 - val_loss: 0.3377 - val_accuracy: 0.9268
Epoch 12/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0075 - accuracy: 0.9980 - val_loss: 0.3612 - val_accuracy: 0.9281
Epoch 13/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0077 - accuracy: 0.9983 - val_loss: 0.3849 - val_accuracy: 0.9250
Epoch 14/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0067 - accuracy: 0.9976 - val_loss: 0.3946 - val_accuracy: 0.9277
Epoch 15/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0082 - accuracy: 0.9972 - val_loss: 0.2622 - val_accuracy: 0.9448
Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0071 - accuracy: 0.9973 - val_loss: 0.6699 - val_accuracy: 0.8847
Epoch 17/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0050 - accuracy: 0.9986 - val_loss: 0.4736 - val_accuracy: 0.9105

Epoch 19/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0032 - accuracy: 0.9990 - val_loss: 0.4601 - val_accuracy: 0.9304
Epoch 20/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0069 - accuracy: 0.9970 - val_loss: 0.4643 - val_accuracy: 0.9299



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 32
Adding Convolution: Kernel Size: 12, Filter Count: 32
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 7ms/step - loss: 0.6511 - accuracy: 0.6186 - val_loss: 0.4660 - val_accuracy: 0.7889
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-235401/assets
Epoch 2/20
288/288 [==============================] - 2s 6ms/step - loss: 0.3921 - accuracy: 0.8297 - val_loss: 0.2997 - val_accuracy: 0.8653
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-235401/assets
Epoch 3/20
288/288 [==================

Epoch 4/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0660 - accuracy: 0.9781 - val_loss: 0.2241 - val_accuracy: 0.9227
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-235515/assets
Epoch 5/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0287 - accuracy: 0.9912 - val_loss: 0.2478 - val_accuracy: 0.9236
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-235515/assets
Epoch 6/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0146 - accuracy: 0.9969 - val_loss: 0.1634 - val_accuracy: 0.9566
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-235515/assets
Epoch 7/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0115 - accuracy: 0.9984 - val_loss: 0.3803 - val_accuracy: 0.9055
Epoch 8/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0146 - accuracy: 0.9952 - val_loss: 0.3038 - val_accuracy: 0.9263
Epoch 9/20
288/288 [==============================] - 2s 6ms/step

INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-235630/assets
Epoch 10/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0385 - accuracy: 0.9867 - val_loss: 0.5040 - val_accuracy: 0.8951
Epoch 11/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0137 - accuracy: 0.9942 - val_loss: 0.3423 - val_accuracy: 0.9277
Epoch 12/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0063 - accuracy: 0.9978 - val_loss: 0.4357 - val_accuracy: 0.9236
Epoch 13/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0090 - accuracy: 0.9970 - val_loss: 0.4206 - val_accuracy: 0.9218
Epoch 14/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0092 - accuracy: 0.9972 - val_loss: 0.2836 - val_accuracy: 0.9467
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-235630/assets
Epoch 15/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0119 - accuracy: 0.9962 - val_loss: 0.2958 - val_accuracy:

Epoch 18/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0101 - accuracy: 0.9965 - val_loss: 1.1358 - val_accuracy: 0.8590
Epoch 19/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0094 - accuracy: 0.9967 - val_loss: 0.2405 - val_accuracy: 0.9598
Epoch 20/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0058 - accuracy: 0.9987 - val_loss: 0.4328 - val_accuracy: 0.9381



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 96
Adding Convolution: Kernel Size: 12, Filter Count: 96
Adding Convolution: Kernel Size: 16, Filter Count: 96
Epoch 1/20
288/288 [==============================] - 2s 7ms/step - loss: 0.5960 - accuracy: 0.6765 - val_loss: 0.4623 - val_accuracy: 0.7776
INFO:tensorflow:Assets written to: ../models/CNNBal17/210408-235822/assets
Epoch 2/20
288/288 [==============================] - 2s 6ms/step - loss: 0.3136 - accuracy: 0.8686 - val_loss: 





### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/